<a href="https://colab.research.google.com/github/dongyoung0/Final_Project/blob/develop/1.processing_code/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 54.7 MB/s 


In [ ]:
from konlpy.tag import Kkma, Okt
from konlpy.utils import pprint

# 텍스트 전처리 🐶

In [ ]:
df = pd.read_csv('data/content.csv')
text_df = pd.read_csv('text_example.csv')

## 필요 함수 정의

### 정규표현식 - 한글 제외 모두 제거

In [ ]:
import re 

def re_kor(s, target = 'list'):
    # pre = re.compile('[^ ㄱ-ㅣ가-힣]')
    pre = re.compile('[^ 가-힣]')
    result = pre.sub(' ', s) 
    while '  ' in result:
        result = result.replace('  ', ' ')
    if target == 'list':
        result = result.split(' ')
        result = [r for r in result if r!='']
    return result

### 단어 개수 세기

In [ ]:
def get_voca_cnt(voca_list):
    total_vocabs = []
    for t in voca_list:
        total_vocabs.extend(t)
    total_unique, total_count = np.unique(total_vocabs, return_counts=True)
    vocab_cnt = pd.DataFrame({'name': total_unique, 'cnt': total_count})
    return vocab_cnt.sort_values('cnt', ascending=False)

## Text_df(예시) 전처리

### 제목 : 정규표현식으로 한글만 남기기

In [ ]:
name_list = text_df.name.apply(re_kor)
name_list

0                          [스튜디오, 톰보이, 패딩]
1                      [키르시, 리버시블, 패딩, 블랙]
2                                  [아이더패딩]
3                                 [뱅뱅겨울야상]
4                        [그레그노먼, 구스다운, 정품]
                       ...                
8513    [톰브라운, 밀라노, 스티치, 사선, 금장, 가디건, 네이비]
8514             [시리즈, 에피그램, 가디건, 사이즈, 니트]
8515           [헤리티지플로스, 가디건, 라이트그레이, 새상품]
8516            [구매글, 안데르센, 안데르센, 스키퍼, 블랙]
8517         [버버리, 레트로, 클래식, 로고, 네이비, 가디건]
Name: name, Length: 8518, dtype: object

### 제목 단어모음 만들기

In [ ]:
name_cnt = get_voca_cnt(name_list)
name_cnt

,name,cnt
4142,자켓,938
4808,코트,854
5075,투피스,848
2681,셔츠,833
2527,새상품,549
...,...,...
2478,삭스,1
2479,삭스블루,1
2481,산드,1
2483,산드로마쥬샤넬비비안웨스트우드,1


### 키워드 : "," 기준으로 구분해주기

In [ ]:
keyword_list = text_df.keyword.fillna('').apply(lambda x: x.split(','))
keyword_list

0                [스튜디오톰보이, 노스페이스, 뉴발란스, 숏패딩, 패딩]
1                     [키르시, 키르시패딩, 미스치프, 미스치프패딩]
2                                    [롱패딩, 패딩점퍼]
3                  [90, 겨울잠바, 겨울점퍼, 여성패딩, 덕다운자켓]
4                                           [95]
                          ...                   
8513    [톰브라운, 톰브라운가디건, 밀라노스티치금장, 가디건, 톰브라운3사이즈]
8514                                    [사이즈100]
8515                                         [S]
8516                [안데르센, 안데르센안데르센, L, XL, 스키퍼]
8517          [버버리100, 버버리가디건, 버버리, 가디건100, 100]
Name: keyword, Length: 8518, dtype: object

### 제목 단어모음 만들기

In [ ]:
keyword_cnt = get_voca_cnt(keyword_list)
keyword_cnt

,name,cnt
7635,투피스,705
7246,코트,531
6452,자켓,528
4454,셔츠,373
2581,롱코트,284
...,...,...
4754,스티치,1
4753,스트링투피스,1
4752,스트링코트,1
4751,스트링스커트,1


## 제목-키워드 단어 모음 만들기

In [ ]:
total_vocab = pd.merge(name_cnt, keyword_cnt,how='outer', on='name').fillna(0)
total_vocab['cnt'] = (total_vocab['cnt_x'] + total_vocab['cnt_y']).astype(int)
total_vocab = total_vocab[['name', 'cnt']]
total_vocab

,name,cnt
0,자켓,1466
1,코트,1385
2,투피스,1553
3,셔츠,1206
4,새상품,595
...,...,...
11989,스티치자켓,1
11990,스티치원피스,1
11991,스트링투피스,1
11992,스트링스커트,1


### 3번 이상 언급된 단어만 ?

In [ ]:
total_vocab= total_vocab[total_vocab.cnt>2]

### 카테고리

일단 카테고리 2만... 카테고리 3도 처리해주면 좋음

In [ ]:
cat_list = text_df.content_cat_2.apply(lambda x: [cat.replace(' ', '') for cat in x.split('/')])

## Description : Kkma, re 😢

In [ ]:
kkma = Kkma()

In [ ]:
# kkma로 명사 추출 후 단어 모음에 있는 단어만 남기기
def kkma_voca(sentence):
    voca_list = kkma.nouns(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# 정규표현식으로 전처리 후 단어 모음에 있는 단어만 남기기
def re_voca(sentence):
    voca_list = re_kor(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# kkma, 정규표현식으로 나온 각 리스트 합쳐서 한번에
def re_kkma_voca(sentence):
    voca_list1 = kkma.nouns(sentence)
    voca_list2 = re_kor(sentence)
    voca_list = voca_list1 + voca_list2
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

- 상세설명 결측 채우기

In [ ]:
text_df.description = text_df.description.fillna('!')

- 전처리 진행

In [ ]:
# kkma 전처리
desc_list = text_df.description.apply(kkma_voca)
# # kkma + re 전처리
# desc_list = text_df.description.apply(re_kkma_voca)

# Word2Vec 학습

### 각 컨텐츠마다 한 문장 만들기(이름 + 키워드 + 상세설명 + 카테고리)

In [ ]:
total_sentences = name_list + keyword_list + desc_list + cat_list
total_sentences

0       [스튜디오, 톰보이, 패딩, 스튜디오톰보이, 노스페이스, 뉴발란스, 숏패딩, 패딩,...
1       [키르시, 리버시블, 패딩, 블랙, 키르시, 키르시패딩, 미스치프, 미스치프패딩, ...
2       [아이더패딩, 롱패딩, 패딩점퍼, 만, 원, 구매, 드라이, 번, 상태, 옷, 패딩...
3       [뱅뱅겨울야상, 90, 겨울잠바, 겨울점퍼, 여성패딩, 덕다운자켓, 사이즈, 90,...
4       [그레그노먼, 구스다운, 정품, 95, 여성, 여성의류, 상품, 정품, 사이즈, 9...
                              ...                        
8513    [톰브라운, 밀라노, 스티치, 사선, 금장, 가디건, 네이비, 톰브라운, 톰브라운가...
8514    [시리즈, 에피그램, 가디건, 사이즈, 니트, 사이즈100, 시리즈, 제품, 숄, ...
8515    [헤리티지플로스, 가디건, 라이트그레이, 새상품, S, 새, 새상품, 상품, 사이즈...
8516    [구매글, 안데르센, 안데르센, 스키퍼, 블랙, 안데르센, 안데르센안데르센, L, ...
8517    [버버리, 레트로, 클래식, 로고, 네이비, 가디건, 버버리100, 버버리가디건, ...
Length: 8518, dtype: object

In [ ]:
print(text_df.iloc[0])
print(total_sentences[0])

pid                                                      160963708
name                                                   스튜디오 톰보이 패딩
keyword                                  스튜디오톰보이,노스페이스,뉴발란스,숏패딩,패딩
description      스튜디오 톰보이 패딩입니다\n\n색상은 네이비 색상이고\n\n남녀공용 프리사이즈 이...
content_cat_1                                                 여성의류
content_cat_2                                                패딩/점퍼
content_cat_3                                            기타(패딩/점퍼)
Name: 0, dtype: object
['스튜디오', '톰보이', '패딩', '스튜디오톰보이', '노스페이스', '뉴발란스', '숏패딩', '패딩', '스튜디오', '톰보이', '패딩', '색상', '네이비', '남녀공용', '공용', '프리', '프리사이즈', '사이즈', '3', '번', '착용', '드라이', '드라이클리닝', '상태', '코듀로이', '코듀로이', '숏', '점퍼', '원가', '만', '원', '포함', '가격', '패딩', '점퍼']


## Word2Vec 학습

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = total_sentences, size = 100, window = 5, min_count = 1, workers = 4, sg = 1)

## 유사도 확인

In [ ]:
print(name_list[0])
print(keyword_list[0])
model.wv.n_similarity(name_list[0], keyword_list[0])

['스튜디오', '톰보이', '패딩']
['스튜디오톰보이', '노스페이스', '뉴발란스', '숏패딩', '패딩']


0.88625646

In [ ]:
i, j = 130, -8
print(name_list.iloc[i])
print(keyword_list.iloc[j])
model.wv.n_similarity(name_list.iloc[i], keyword_list.iloc[j])

['맥케이지', '여성패딩', '사이즈']
['꼼데가르송', '꼼데가르송가디건', '꼼데가르송반팔', '꼼데가르송컨버스', '꼼데가르송셔츠']


0.83096015

In [ ]:
for i in range(50):
    print(name_list.iloc[8104])
    print(keyword_list.iloc[i])
    print(desc_list.iloc[i])
    print('제목-키워드', model.wv.n_similarity(name_list.iloc[8104], keyword_list.iloc[i]))
    try:
        print('제목-설명', model.wv.n_similarity(name_list.iloc[8104], desc_list.iloc[i]))
    except:
        pass
    print('----------------------------------------------------------------------------')


## FastText 😺

새로운 단어에 대응 가능

In [ ]:
from gensim.models import FastText
model_ft = FastText(size=100, window=3, min_count=5)
model_ft.build_vocab(sentences=total_sentences)
model_ft.train(sentences=total_sentences, total_examples=len(total_sentences), epochs=10)

In [ ]:
print(name_list[0])
print(keyword_list[0])
model_ft.wv.n_similarity(name_list[0], keyword_list[0])

['스튜디오', '톰보이', '패딩']
['스튜디오톰보이', '노스페이스', '뉴발란스', '숏패딩', '패딩']


0.858155

In [ ]:
print(name_list[0])
print(keyword_list[4000])
model_ft.wv.n_similarity(name_list[0], keyword_list[4000])

['스튜디오', '톰보이', '패딩']
['free']


0.49088758

In [ ]:
model_ft.wv.similarity('나이키', '아디다스')

0.98482955

In [ ]:
model_ft.wv.n_similarity(['아디다스', '패딩'], ['꼼데가르송', '가디건'])

0.6543265

In [ ]:
model_ft.wv.n_similarity(['아디다스패딩나이키'], ['나이키', '아디다스', '패딩'])

0.942635